In [1]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
model.eval()
embeddings = model.encode(sentences)
print(embeddings)



Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

[[-0.03375624 -0.06316338 -0.0316612  ...  0.03684864 -0.02036646
  -0.01574   ]
 [-0.01409588  0.00091114 -0.00096315 ... -0.02571585 -0.00289072
  -0.00579975]]


In [5]:
embeddings = model.encode(sentences)
print(embeddings.shape)

(2, 768)


In [9]:
model.parameters()

<generator object Module.parameters at 0x7f6de15ff200>

In [36]:
import json


all_spans = {}

with open("../../mfc_v4.0/spans_no_context.json", "r") as f:
    all_spans = json.load(f)

In [37]:
encoded_input

{'input_ids': tensor([[    0,   713,    16,    41,  1246,  3645,     2],
        [    0, 20319,  3645,    16,  8417,     2,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}

In [42]:
import torch
import torch.nn as NN


class SentenceClassifier(NN.Module):
    def __init__(self, labels):
        
        super().__init__()
        
        self.transformer = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
        for params in self.transformer.parameters():
            params.requires_grad = False
        
        self.fc = NN.Linear(768, len(labels))
        self.logits = NN.Softmax()
        self.labels = labels
    
    def forward(self, x):
        return self.logits(self.fc(torch.tensor(self.transformer.encode(x))))
    




# x = []
# y = []

# count = 0

# n_spans = sum(len(all_spans[key]) for key in all_spans)

# batch_x = []
# batch_y = []



# with torch.no_grad():
#     for label in all_spans:
#         if count > 100:
#             break
#         for span in all_spans[label]:
#             count += 1
#             batch_x.append(span)
#             batch_y.append(label)
#             if len(batch_x) == 60:
#                 tokenized = tokenizer(batch_x, return_tensors='pt', padding=True)
#                 output = model(**tokenized)
#                 x.append(output.last_hidden_state)
#                 y.append(batch_y)
#                 batch_x = []
#                 batch_y = []
#             if count % 100 == 0:
#                 print("Embedding for span %d of %d computed" % (count, n_spans))
#                 print(x[-1].shape)

In [62]:
data = []

for i,key in enumerate(all_spans):
    for span in all_spans[key]:
        data.append((span, i))

import random

random.shuffle(data)

train = data[:200]
test = data[200:300]

train_x = [d[0] for d in train]
train_y = [[1 if i == d[1] else 0 for i in range(len(all_spans.keys()))] for d in train]
train_y = torch.tensor(train_y, dtype=torch.float)
test_x = [d[0] for d in test]
test_y = [[1 if i == d[1] else 0 for i in range(len(all_spans.keys()))] for d in test]
test_y = torch.tensor(test_y, dtype=torch.float)

In [67]:
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 0.01

model = SentenceClassifier(list(all_spans.keys()))
model.train()


optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(20):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(train_x)

    # Compute and print loss.
    loss = loss_fn(y_pred, train_y)
    print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

/home/sam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 195.54608154296875
1 175.13975524902344
2 144.00485229492188
3 114.10669708251953
4 91.95745849609375
5 76.05500793457031
6 65.9549560546875
7 58.249267578125
8 52.108787536621094
9 47.50080871582031
10 44.1169319152832
11 41.495079040527344
12 39.436214447021484
13 37.789634704589844
14 36.43472671508789
15 35.241798400878906
16 33.92860412597656
17 32.567955017089844
18 31.745241165161133
19 31.02952766418457


In [75]:
test = model("The fed is planning on raising rates")

/home/sam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [76]:
labels = list(all_spans.keys())

pred = int(torch.argmax(test))

labels[pred]

'1.0'